
---
<big><big><big><big><big><big><big><big><big>Sieci neuronowe</big></big></big></big></big></big></big>

---



---
<big><big><big><big><big>Autoenkoder</big></big></big></big></big>

---



---

<id=tocheading><big><big><big><big>Spis treści</big></big></big></big>
<div id="toc"></div>

---

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [2]:
# -*- coding: utf-8 -*-

from __future__ import unicode_literals
from __future__ import print_function
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bokeh.charts import Scatter

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
sns.set(font_scale=2.0)

# Autoenkoder

to model (najczęściej sieć neuronowa), który uczy się __odtwarzać__ podawane wejście
1. __enkoder__ $$h=f(x)$$
2. __dekoder__ $$r=g(h)$$


Model, który uczy się idealnie tak, że $$g(f(x))=x$$ __nie jest__ specjalnie przydatny

Autoenkodery wykorzystują założenie, że dane są skupione na niżej wymiarowych rozmaitościach

## PCA - Principal Components Analysis
* przykład prostego autoenkodera
* PCA przetwarza wektor wejściowy $x$ na __przybliżoną__ rekonstrukcję $r$ wykorzystując __liniową__ transformację $$W^TWx$$
<img src="../mum_figures/pca.png" width="60%"/>[za Ivizevic _et al._]
  * dowolny obrót zachowuje względne relacje danych
  * wybieramy taki obrót, który zmaksymalizuje łatwość rozróżniania danych
    * maksymalizacja wariancji wzdłuż wynikowych kierunków głównych (_ang_. principal components)
    * równoważne regresji __minimalizującej kwadraty odległości__ od składowych głównych
    
    
    
* w jaki sposób spojrzeć na PCA jako autoenkoder?
  1. __enkoder__ odwzorowuje $X$ w $H$
    * to odwzorowanie z $X$ do przestrzeni o kształcie _naleśnika_ (Bengio, Goodfellow)
    * część współrzędnych, o najmniejszej wariancji, jest całkiem płaska i może być usunięta
  2. __dekoder__ odwzorowuje $H$ z powrotem do $X$
    * część informacji jest utracona
      * ale pewnie związana z szumem


* można spojrzeć jak na __manifold learning__

## Wymiar $H$ mniejszy od wymiaru $X$
* wymuszenie, by zachować najważniejsze informacje
* uczenie przez minimalizację $$L(x, g(f(x)))$$
  * jeśli dekoder liniowy a $L$ średnim błędem kwadratowym, to model równoważny PCA
* autoenkodery z __nieliniowymi__ enkoderem i dekoderem są zatem modelami __silniejszymi__
  * nieliniowe funkcje mają większą pojemność od liniowych
  * jednak zbyt złożony i pojemny model może nie nauczyć się niczego!

## Regularyzacja
* modele zbyt pojemne mogą się nie nauczyć niczego
* przestrzeń __ukryta__ (_ang_. __latent__) $H$
* wymiar $H$ może być większy od wejściowego
  * nawet liniowe enkoder i dekoder mogą nie nauczyć się niczego
    * będą umiały kopiować wejście, ale żadnej wiedzy o __strukturze__ wejścia
  * __regularyzacja__ modelu zamiast ograniczeń na wymiar i postać en-/de-koderów wymuszające
    * rzadkość reprezentacji
    * odporność na szum wejścia
    * odporność na brakujące dane
    * niską pochodną reprezentacji
    
    
* także modele __generatywne__
  * autoenkoder __wariacyjny__
  * sieci __stochastyczne__

## Autoenkoder z rzadką reprezentacją
* głównym zadaniem jest __ wykrywanie__ (reakcja na) istotnych cech w wejściu
* uczenie z

$$L(x, g(f(x)))+\Omega(h)$$
gdzie $\Omega(h)$ jest regularyzatorem dodanym do sieci 
  * kopiującej w uczeniu nienadzorowanym swoje wejście
  * wykrywającej cechy w uczeniu nadzorowanym
  
* model będzie maksymalizował

$$\log\,p_m(x, h)=\log\,p_m(h)+\log_m\,p_m(x|h)$$
  * w ten sposób jest to model __maksymalizujący__ sznasę odzyskania $x$ przy założeniu pewnych ograniczeń na zmienne ukryte $h$
   * warunek $\log\,p_m(h)$ może wymuszać __rzadką__ reprezentację, np. 
   
   $$p_m(h_i)=\frac{\lambda}{2}\exp(-\lambda\,|\,h_i\,|)$$
   
* rzadka reprezentacja
  * nie jest czynnikiem regularyzacyjnym, ale
  * wymusza uczenie zmiennych ukrytych, które __tłumaczą__ wejście
  
  
  
* użycie $\Omega(h)$ jak wyżej
  * nie zapewnia ścisłej rzadkiej reprezentacji
  * uzyskanie rzeczywistych zer możliwe przez użycie dodatkowo aktywacji ReLU
    * pozwala uzyskać roprezentację z ustaloną liczbą zer
    * możliwe użycie _softmax_ na warstwie ukrytej

## Autoenkodery odszumiające
* użycie kosztu

$$L(x, g(f(\tilde{x})))$$
gdzie $\tilde{x}$ jest zaszumionym wejściem
* zadaniem jest _usuwanie_ szumu
  * domyślnie uczy się struktury danych
  <img src="../mum_figures/DAE.png" width="100%"/>[za Bengio _et al_.]
  
  
  
1. losowanie przykładu $x$ ze danych uczących
2. losowanie danych zaburzonych $\tilde{x}$ z $Z(\tilde{x}\mid x)$
3. użycie $(\tilde{x},x)$ jako danej uczącej
  * odtwarzanie z $$p(x\mid\tilde{x})=p(x\mid h)$$
  gdzie zmienna ukryta $h=f(\tilde{x})$
4. dekoder $g(h)$


* uczenie gradientowe przez minimalizację log-likelihood $-\log\,p(x\mid h=f(\tilde{x}))$

## Uczenie rozmaitości
Autoenkodery mają zapewniać __jednocześnie__
1. uczenie reprezentacji $h=f(x)$ takiej, że
  * $x$ można odtworzyć
  * dane losowane są z danych uczących
    * autoenkoder nie musi poprawnie odtwarzać danych spoza rozkładu
2. zapewnienie ograniczeń
  * ograniczenia architektury ograniczające pojemność
  * czynniki regularyzacyjne ograniczające wrażliwość na szum
  
  

* reprezentacja ukryta opisuje strukturę generującą dane
  * enkoder tworzy reprezentację odwzorowującą geometrię nisko-wymiarowej rozmaitości gdzie znajdują się dane
  * samo kopiowanie wejścia __nie jest__ istotne
  
  <img src="../mum_figures/manifold.png" width="100%"/>
* metody uczenia manifoldów oparte na metodach sąsiadów
  * tworzą graf sąsiadów
  * jeśli przykładów jest wystarczająco wiele
    * pojedyncze przykłady można uogólnić do "płaskich" gausianów
    * to pozwoli odtworzyć geometrię rozmaitości
    * możliwe będzie generowanie nowych danych przez uśrednianie między obszarami przykładów
* jeśli rozmaitości __nie są__ gładkie
  * będzie potrzebna duża liczba przykładów w okolicach każdego zaburzenia gładkości
  * generowanie będzie utrudnione
    * możliwe jedynie pomiędzy bliskimi sąsiadami

* powinno pomóc uzycie głębokich autoenkoderów

# Autoenkoder wariacyjny
* autoenkodery znajdują odwzorowanie $x\longrightarrow\,h$
* model wariacyjny pozwala na odwzorowanie $x$ w __rozkład__ $\mathcal{N}(\mu,\sigma^2)$ w przestrzeni ukrytej
  * to pozwala na __próbkowanie__ z rozkładu normalnego
    * model losuje z rozkładu $p_m(z)$
    * tworzy $x'=g(z)$
* model wariacyjny __maksymalizuje__

$$L(q)=E_{z\tilde\,q(z\mid x)}\log\,p_m(z,x)+H(q(z\mid x))$$
  * model składa się z __enkodera__ $q(z\mid x)$ tworzącego $z$
  * oraz __dekodera__ $p_m(x\mid z)$
  * pierwszy składnik maksymalizuje całkowite prawdopodobieństwo zmiennych ukrytych i widzialnych
  * drugi jest entropią $q$
    * $q$ jest (może być) zwykle gausowski
    * maksymalizacja entropii zachęca model do skupianiu się na __wielu__ wartościach $z$, które mogą wygenerować $x$, a nie tylko na jednej
    * to jest odmienne od modelu mapowania punkt w punkt


* autoenkoder wariacyjny może być łatwo nauczony
  * możliwe jest wiele architektur enkodera i dekodera
    * wszystkie warstwowe
    * enkoder warstwowy a dekoder rekurencyjny
    * obie rekurencyjne

---
<img src="../mum_figures/vae-faces.png" width="100%"/>[za Bengio et al]
* twarze na rysunku __wygenerowane__
* przestrzeń ukryta w 2 wymiarach
  * oś pozioma wyraźnie odpowiada obrotowi
  * oś pionowa wyrazowi twarzy
---
    
* wygenerowane przykłady z nauczonego autoenkodera mają tendencję by być nieco _zamazane_ (_ang_. blurry) [Bengio _et al_.]
  * nie są znane jeszcze powody